In [1]:
%env MKL_NUM_THREADS=4
%env MKL_DEBUG_CPU_TYPE=5
%env THEANO_FLAGS=device=cpu, floatX=float32

import pickle
import pandas as pd
import numpy as np
import pymc3 as pm
import arviz as az
import theano.tensor as tt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from collections import Counter
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(level=logging.CRITICAL)

plt.rcParams['font.family'] = 'Nimbus Sans'

INPUT_DATA_AUX_STATS = '../data preparation/all_patients_firstDiag_03292021_otherStats.bpkl3'
TIME_DIFF_DATA = '../data preparation/two_dis_time_diff_seqs_03312021.bpkl3'
OUTCOME_DIS = 'Schizophrenia_Related_Psychosis'
# INTERVENTION = 'Abnormal_Spine_Curvature'
TIME_DIFF_UNIT = 100
TIME_DIFF_MIM_THRESHOLD = 100
NUM_CHAINS = 4

HDI_PROB = 0.95

with open(INPUT_DATA_AUX_STATS, 'rb') as f:
    all_aux_stats = pickle.load(f)
    DIS_IDX_DICT = all_aux_stats['dis_to_idx']
    DIS_SYS_DICT = all_aux_stats['dis_to_sys']
    
with open(TIME_DIFF_DATA, 'rb') as f:
    ALL_DIS_PAIR_TIME_DIFF = pickle.load(f)

dis_name_list = []
shift_mean_list = []
shift_hdi_list = []
shift_sig_list = []
alter_waic_list = []
null_waic_list = []
alter_better_list = []
for k, v in DIS_IDX_DICT.items():
    
    INTERVENTION = k
    TIME_DIFF = ALL_DIS_PAIR_TIME_DIFF[(OUTCOME_DIS, 
                                        INTERVENTION, 
                                        DIS_IDX_DICT[OUTCOME_DIS],
                                        DIS_IDX_DICT[INTERVENTION])]
    
    print('\033[94m'+OUTCOME_DIS+' x '+INTERVENTION+'\033[0m')
    
    if (len(TIME_DIFF)) < TIME_DIFF_MIM_THRESHOLD or (INTERVENTION == OUTCOME_DIS):
        continue
        
    dis_name_list.append(INTERVENTION)
    
    data_out_path = f'RDD_experiment_{OUTCOME_DIS}_06202022/{INTERVENTION}'
    
    with open(data_out_path+'/analysis_results_and_plot_data.bpkl3', 'rb') as f:
        rdd_result = pickle.load(f)
        alter_trace = rdd_result['alter.trace']
        alter_waic = rdd_result['alter.waic']
        null_trace = rdd_result['null.trace']
        null_waic = rdd_result['null.waic']
        shift_alter_trace = alter_trace['shift']
    
    shift_mean = shift_alter_trace.mean()
    shift_hdi = az.hdi(shift_alter_trace, hdi_prob=HDI_PROB)
    shift_sig = (shift_hdi[0] * shift_hdi[1] > 0.0)
    
    shift_mean_list.append(shift_mean)
    shift_hdi_list.append(list(shift_hdi))
    shift_sig_list.append(shift_sig)
    
    alter_waic_list.append(alter_waic.waic)
    null_waic_list.append(null_waic.waic)
    alter_better_list.append(alter_waic.waic<null_waic.waic)

env: MKL_NUM_THREADS=4
env: MKL_DEBUG_CPU_TYPE=5
env: THEANO_FLAGS=device=cpu, floatX=float32
Schizophrenia_Related_Psychosis x ADHD
Schizophrenia_Related_Psychosis x Abnormal_Spine_Curvature
Schizophrenia_Related_Psychosis x Acne
Schizophrenia_Related_Psychosis x Acquired_Coagulation_Defect
Schizophrenia_Related_Psychosis x Acquired_Hemolytic_Anemias
Schizophrenia_Related_Psychosis x Acquired_Hypothyroidism
Schizophrenia_Related_Psychosis x Acquired_Limb_Deformities
Schizophrenia_Related_Psychosis x Acquired_Other_Myopathies
Schizophrenia_Related_Psychosis x Acquired_Retinal_Defects
Schizophrenia_Related_Psychosis x Acquired_Visual_Disturbances
Schizophrenia_Related_Psychosis x Acrocephalosyndactyly
Schizophrenia_Related_Psychosis x Acute_Bronchitis
Schizophrenia_Related_Psychosis x Acute_Glomerulonephritis
Schizophrenia_Related_Psychosis x Acute_Renal_Failure
Schizophrenia_Related_Psychosis x Acute_Sinusitis
Schizophrenia_Related_Psychosis x Acute_Upper_Respiratory_Infection
Schizoph

In [2]:
rdd_result_df = pd.DataFrame({'Intervention': dis_name_list,
                              'Shift mean': shift_mean_list,
                              'Shift HDI': shift_hdi_list,
                              'Shift HDI significance': shift_sig_list,
                              'Shifted model WAIC': alter_waic_list,
                              'Non-shifted model WAIC': null_waic_list,
                              'Shifted mdodel better?': alter_better_list})

In [3]:
rdd_result_df.to_csv('RDD_experiment_Schizophrenia_06202022_linearTrend_result.csv')
rdd_result_df.sort_values(by=['Shift mean'])

,Intervention,Shift mean,Shift HDI,Shift HDI significance,Shifted model WAIC,Non-shifted model WAIC,Shifted mdodel better?
11,Gastrointestinal_Infection,0.802432,"[-1.1170601, 3.1404104]",False,160.060370,159.500837,False
17,Non-Specific_Pulmonary_Disorder,0.818385,"[-0.81348085, 2.9494545]",False,160.576344,160.224561,False
18,Pneumonia,0.834401,"[-0.98809916, 2.8608396]",False,186.312030,189.490307,True
3,Allergic_Rhinitis,1.099511,"[-0.87704706, 3.7586193]",False,211.808730,210.826497,False
12,General_Ear_Disorder,1.115376,"[-0.33882678, 2.5548854]",False,148.432409,155.127621,True
4,Asthma,1.223736,"[-1.2983358, 4.3391695]",False,190.338511,190.414548,True
9,Functional_Digestive_Disorder,1.614734,"[-1.3742731, 4.6153197]",False,169.749770,169.421240,False
0,Acute_Bronchitis,1.705400,"[-1.7592145, 5.615645]",False,236.931001,234.327623,False
8,Eye_Infection,2.529403,"[-0.83097637, 6.2364335]",False,211.201226,209.192314,False
10,Fungal_Infection,2.581091,"[-0.67397946, 5.910047]",False,163.124174,168.203396,True


In [4]:
rdd_result_df_sig = rdd_result_df.loc[rdd_result_df['Shift HDI significance']==True]
rdd_result_df_sig = rdd_result_df_sig.sort_values(by=['Shift mean'])
rdd_result_df_sig.reset_index(inplace=True)
rdd_result_df_sig.to_csv('RDD_experiment_Schizophrenia_06202022_linearTrend_sig_result.csv')
rdd_result_df_sig

,index,Intervention,Shift mean,Shift HDI,Shift HDI significance,Shifted model WAIC,Non-shifted model WAIC,Shifted mdodel better?
0,7,Esophageal_Disease,3.203482,"[0.8483902, 5.7468276]",True,158.898282,168.874633,True
1,16,Non-Specific_Gastrointestinal_Disorder,3.731916,"[1.5988384, 6.247144]",True,177.926569,186.535657,True
2,2,Acute_Upper_Respiratory_Infection,9.815643,"[2.1563118, 15.6782465]",True,288.014012,284.437655,False
